# 第九章 创建和部署远程服务器

在之前的课程中，我们学习了使用 `stdio` 传输在本地运行服务器。在本课程中，我们将学习如何使用 `FastMCP` 创建一个远程服务器，并使用 MCP inspector 进行测试，然后学习如何将其部署到 `render.com`。
我们将重点关注 `sse` 传输。除此之外，我们包含了如何使用 `Streamable HTTP` 实现服务器的一些内容。

## 一、创建远程服务器

通过 `FastMCP` 可以轻松创建一个SSE远程服务器。我们只需要在运行服务器时指定传输为 `sse`。我们还可以在初始化 `FastMCP` 服务器时指定端口号。

在开始之前，请安装以下依赖：
```sh
uv pip install mcp arxiv
```

接下来创建本地文件夹，将research_server.py文件保存到该文件夹中。


In [1]:
%mkdir -p mcp_project

运行以下代码，将代码保存到mcp_project/research_server.py文件中。

In [2]:
%%writefile mcp_project/research_server.py
import arxiv
import json
import os
from typing import List
from mcp.server.fastmcp import FastMCP

PAPER_DIR = "papers"

# 初始化服务器，创建一个名为 research 的 server，并监听 8001 端口
mcp = FastMCP("research", port=8001)

@mcp.tool()
def search_papers(topic: str, max_results: int = 5) -> List[str]:
    """
    根据一个主题搜索arXiv上的论文，并存储他们的信息。
    
    参数：
    - topic: 要搜索的主题
    - max_results: 最大结果数（默认：5）
    
    返回：
    - 找到的论文ID列表
    """
    
    # 使用 arxiv 搜索论文
    client = arxiv.Client()

    # 搜索与查询主题最相关的文章
    search = arxiv.Search(
        query = topic,
        max_results = max_results,
        sort_by = arxiv.SortCriterion.Relevance
    )

    papers = client.results(search)
    
    # 创建目录用于这个主题
    path = os.path.join(PAPER_DIR, topic.lower().replace(" ", "_"))
    os.makedirs(path, exist_ok=True)
    
    file_path = os.path.join(path, "papers_info.json")

    # 尝试加载已有的论文信息
    try:
        with open(file_path, "r") as json_file:
            papers_info = json.load(json_file)
    except (FileNotFoundError, json.JSONDecodeError):
        papers_info = {}

    # 处理每个论文并添加到 papers_info
    paper_ids = []
    for paper in papers:
        paper_ids.append(paper.get_short_id())
        paper_info = {
            'title': paper.title,
            'authors': [author.name for author in paper.authors],
            'summary': paper.summary,
            'pdf_url': paper.pdf_url,
            'published': str(paper.published.date())
        }
        papers_info[paper.get_short_id()] = paper_info
    
    # 将更新后的 papers_info 保存到 json 文件
    with open(file_path, "w") as json_file:
        json.dump(papers_info, json_file, indent=2)
    
    print(f"Results are saved in: {file_path}")
    
    return paper_ids

@mcp.tool()
def extract_info(paper_id: str) -> str:
    """
    从papers目录中提取指定论文ID的详细信息。
    
    参数：
    - paper_id: 要提取信息的论文ID
    
    返回：
    - 如果找到，返回论文的详细信息（JSON字符串）
    - 如果未找到，返回错误信息
    """
 
    for item in os.listdir(PAPER_DIR):
        item_path = os.path.join(PAPER_DIR, item)
        if os.path.isdir(item_path):
            file_path = os.path.join(item_path, "papers_info.json")
            if os.path.isfile(file_path):
                try:
                    with open(file_path, "r") as json_file:
                        papers_info = json.load(json_file)
                        if paper_id in papers_info:
                            return json.dumps(papers_info[paper_id], indent=2)
                except (FileNotFoundError, json.JSONDecodeError) as e:
                    print(f"Error reading {file_path}: {str(e)}")
                    continue
    
    return f"There's no saved information related to paper {paper_id}."



@mcp.resource("papers://folders")
def get_available_folders() -> str:
    """
    获取所有可用的主题目录。
    
    返回：
    - 所有主题目录的列表
    """
    folders = []
    
    # 获取所有主题目录
    if os.path.exists(PAPER_DIR):
        for topic_dir in os.listdir(PAPER_DIR):
            topic_path = os.path.join(PAPER_DIR, topic_dir)
            if os.path.isdir(topic_path):
                papers_file = os.path.join(topic_path, "papers_info.json")
                if os.path.exists(papers_file):
                    folders.append(topic_dir)
    
    # 创建一个简单的markdown列表
    content = "# Available Topics\n\n"
    if folders:
        for folder in folders:
            content += f"- {folder}\n"
        content += f"\nUse @{folder} to access papers in that topic.\n"
    else:
        content += "No topics found.\n"
    
    return content

@mcp.resource("papers://{topic}")
def get_topic_papers(topic: str) -> str:
    """
    获取特定主题的论文详细信息。
    
    参数：
    - topic: 要检索论文的主题
    """
    topic_dir = topic.lower().replace(" ", "_")
    papers_file = os.path.join(PAPER_DIR, topic_dir, "papers_info.json")
    
    if not os.path.exists(papers_file):
        return f"# No papers found for topic: {topic}\n\nTry searching for papers on this topic first."
    
    try:
        with open(papers_file, 'r') as f:
            papers_data = json.load(f)
        
        # Create markdown content with paper details
        content = f"# Papers on {topic.replace('_', ' ').title()}\n\n"
        content += f"Total papers: {len(papers_data)}\n\n"
        
        for paper_id, paper_info in papers_data.items():
            content += f"## {paper_info['title']}\n"
            content += f"- **Paper ID**: {paper_id}\n"
            content += f"- **Authors**: {', '.join(paper_info['authors'])}\n"
            content += f"- **Published**: {paper_info['published']}\n"
            content += f"- **PDF URL**: [{paper_info['pdf_url']}]({paper_info['pdf_url']})\n\n"
            content += f"### Summary\n{paper_info['summary'][:500]}...\n\n"
            content += "---\n\n"
        
        return content
    except json.JSONDecodeError:
        return f"# Error reading papers data for {topic}\n\nThe papers data file is corrupted."

@mcp.prompt()
def generate_search_prompt(topic: str, num_papers: int = 5) -> str:
    """Generate a prompt for Claude to find and discuss academic papers on a specific topic."""
    return f"""Search for {num_papers} academic papers about '{topic}' using the search_papers tool. 

    Follow these instructions:
    1. First, search for papers using search_papers(topic='{topic}', max_results={num_papers})
    2. For each paper found, extract and organize the following information:
       - Paper title
       - Authors
       - Publication date
       - Brief summary of the key findings
       - Main contributions or innovations
       - Methodologies used
       - Relevance to the topic '{topic}'
    
    3. Provide a comprehensive summary that includes:
       - Overview of the current state of research in '{topic}'
       - Common themes and trends across the papers
       - Key research gaps or areas for future investigation
       - Most impactful or influential papers in this area
    
    4. Organize your findings in a clear, structured format with headings and bullet points for easy readability.
    
    Please present both detailed information about each paper and a high-level synthesis of the research landscape in {topic}."""

@mcp.prompt()
def generate_search_prompt_cn(topic: str, num_papers: int = 5) -> str:
    """生成一个中文提示，用于让Claude搜索关于特定主题的学术论文。"""
    return f"""使用search_papers工具搜索关于'{topic}'的{num_papers}篇学术论文。

    请遵循以下指令：
    1. 首先，使用search_papers(topic='{topic}', max_results={num_papers})搜索论文
    2. 对于每篇找到的论文，提取并组织以下信息：
       - 论文标题
       - 作者
       - 发表日期
       - 关键发现摘要
       - 主要贡献或创新
       - 使用的研究方法
       - 与主题'{topic}'的相关性
       
    3. 提供一个全面的总结，包括：
       - '{topic}'当前研究状态的概述
       - 论文之间的共同主题和趋势
       - 研究差距或未来研究方向
       - 该领域最具影响力或影响力的论文
    
    4. 以清晰、结构化的格式呈现你的发现，使用标题和列表项以便于阅读。

    请同时提供每篇论文的详细信息以及{topic}研究领域的整体概述。
    """
    

if __name__ == "__main__":
    # Initialize and run the server
    mcp.run(transport='sse')

Overwriting mcp_project/research_server.py


我们也可以使用FastMCP创建一个远程服务器，使用"Streamable HTTP"传输。代码与工具、资源和提示定义相同。但是当我们运行服务器时，需要指定传输为：

```python
mcp.run(transport="streamable-http")
```

当初始化FastMCP服务器时，我们有两种选择：

```python
# 有状态服务器（维护会话状态）
mcp = FastMCP("research")

# 无状态服务器（无会话持久化）
mcp = FastMCP("research", stateless_http=True)
```

有状态服务器可以用于处理需要记住客户端信息和上下文的多个请求（例如，处理一个工作流中的多个请求）。无状态服务器可以用于处理简单的独立请求（例如，处理一个请求）。

## 二、测试SSE远程服务器

在我们创建好文件（位于mcp_project/research_server.py）后，我们可以使用MCP inspector或lesson 5中的简单聊天机器人来测试它，也可以将它与lesson 7中的聊天机器人集成。我们首先需要启动以获取其`URL`，然后将`URL`提供给聊天机器人或MCP inspector。

**注意**: 使用`stdio`传输的server是由MCP client作为子进程启动的。而远程server是独立于client运行的进程，需要在client连接到它之前已经运行。

我们首先需要创建一个单独的环境来运行远程服务器。步骤如下：
- 使用`uv init`初始化文件夹，
- 创建一个虚拟环境并激活它，
- 添加所需的依赖（`uv add arxiv mcp`）。

然后，我们可以在终端中运行服务器（`uv run research_server.py`），获得服务器的运行地址。我们需要将这个地址提供给inspector或聊天机器人，在地址末尾添加`/sse`。在第二个终端中，我们需要启动MCP inspector来连接服务器。如果使用Streamable HTPP，则需要在地址末尾添加`/mcp/`。

打开终端，运行以下代码：
```python
python mcp_project/research_server.py
```
获得如下输出：

<img src="images/ch10_launch_example.jpg" weight="300">


In [14]:
# 注意，该代码块仅做示例，如需运行，请额外打开一个terminal然后运行；
# 该代码块运行后会输出服务器地址，之后阻塞，需要手动关闭
!python mcp_project/research_server.py

INFO:     Started server process [40820]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8001 (Press CTRL+C to quit)
^C
INFO:     Shutting down
INFO:     Waiting for application shutdown.


### 三、测试服务器

在获得URL链接之后，我们就可以使用MCP inspector或聊天机器人来测试远程服务器。

打开一个新的terminal，保证之前的继续运行不要中断，运行以下代码：
```bash
npx @modelcontextprotocol/inspector
```
会得到如下所示输出：
```log
Need to install the following packages:
@modelcontextprotocol/inspector@0.14.0
Ok to proceed? (y) y

Starting MCP inspector...
⚙️ Proxy server listening on port 6277
🔍 MCP Inspector is up and running at http://127.0.0.1:6274 🚀

```
打开terminal中弹出的链接。

如下是MCP inspector的界面，我们可以在其中输入提示词，并选择不同的模型来测试。

<img src="images/ch10_inspector2.png" height="300">


我们需要设置以下参数：
1. `Transport Type` 选择 `SSE`
2. `URL` 选择远程服务器地址，该地址以`sse`结尾，是之前运行代码块的输出

本地启动后示例如下图：

<img src="images/ch10_local_example.jpg" height="300">

## 四、资源

- 在CloudFlare上部署远程MCP服务器 [链接](https://developers.cloudflare.com/agents/guides/remote-mcp-server/)
- Streamable HTTP传输 [链接](https://github.com/modelcontextprotocol/python-sdk/blob/main/README.md#streamable-http-transport)
- 对于使用Streamable HTTP实现的低级服务器，请参阅：
    - 有状态服务器：[示例](https://github.com/modelcontextprotocol/python-sdk/tree/main/examples/servers/simple-streamablehttp)
    - 无状态服务器：[示例](https://github.com/modelcontextprotocol/python-sdk/tree/main/examples/servers/simple-streamablehttp-stateless)   